Сбор отзывов с Флампа. Запись отзывов в data/places.csv и data/reviews.csv

In [1]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd

# загружаем env-файл с ключами
load_dotenv() 

# урлы для сбора мест и отзывов
url_places = 'https://flamp.ru/api/2.0/filials/?project=moscow&limit=50&metarubric={metarubric}&nocache=true&page={page}' \
             '&fields=id,rating_decimal,lat,lon,name_primary,name_extension,address,city,basic_attributes'
url_reviews = 'https://flamp.ru/api/2.0/filials/{place_id}/reviews?limit=50&is_trusted=true{offset}'


# хидеры
headers = {
    'X-Application': 'Flamp4',
    'Origin': 'https://moscow.flamp.ru',
    'Authorization': f'Bearer {os.getenv('FLAMP_KEY')}',
    'Accept': ';q=1;depth=0;scopes={};application/json',
    'Referer': 'https://moscow.flamp.ru/',
    'Accept-Encoding': 'gzip, deflate, br'
}

In [8]:
def get_places_by_type(metarubric):
    """Получение мест из флампа с координатами типа metaubric"""
    places = []
    total = True
    page = 1

    while total != 0:
        print(f'Страница {page}')

        request = requests.get(url_places.format(page=page, metarubric=metarubric), headers=headers)
        places.extend([{
            'id': place['id'],
            'name_primary': place['name_primary'],
            'name_extension': place['name_extension'],
            'city': place['city'],
            'address': place['address'],
            'lat': place['lat'],
            'lon': place['lon'],
            'rating_decimal': place['rating_decimal'],
            'avg_price': place['basic_attributes']['avg_price'],
        } for place in request.json()['filials']])

        total = request.json()['meta']['total']
        page += 1

    return places

In [3]:
def get_reviews(place_id):
    """Получение отзывов места по ID флампа"""
    reviews = []
    total = True
    offset = ''

    while total != 0:
        request = requests.get(
            url_reviews.format(place_id=place_id, offset=offset),
            headers=headers).json()

        if 'reviews' in request:
            reviews.extend({
                'id': review['id'],
                'text': review['text'],
                'rating': review['rating'],
                'place_id': place_id # для привязки к месту
            } for review in request['reviews'])

        offset = '&offset_id=%s' % reviews[-1]['id'] if len(reviews) > 0 else ''
        total = len(request['reviews'])

    return reviews

Загрузка датасета по всем типам заведений

In [ ]:
metarubrics = ['restorany', 'bary', 'eda', 'kofejni', 'kafe', 'dostavka-edy', 'fastfud', 'sushi', 'nochnye-kluby']
all_places = sum([get_places_by_type(metarubric=metaub) for metaub in metarubrics], [])
all_places_df = pd.DataFrame(all_places)
all_places_df.to_csv('data/places.csv', sep=';', index=False)

Загрузка датасета по отзывам заведений из all_places

In [ ]:
reviews = sum(map(get_reviews, all_places_df['id'].values), [])
df = pd.DataFrame(reviews)
df.to_csv('data/reviews.csv', sep=';', index=False)